In [ ]:
import pandas as pd
import numpy as np
from itertools import islice
import os

def genotypeOfHaplotypes(genotype_info):
    
    FirstPart = genotype_info.split(":")[0];
    
    if not ("|" in FirstPart):
        return "22";
    
    FirstPartSplit = FirstPart.split("|");
    
    genotype_hap1 = 0 if FirstPartSplit[0] == "0" else 1;
    genotype_hap2 = 0 if FirstPartSplit[1] == "0" else 1;
    return (str(genotype_hap1) + str(genotype_hap2));




maximum_number_varinats = 16
# where individuals id start in vcf file
individual_starting_column = 12 
for variationCount in range(1,maximum_number_varinats):



    lookup_table= pd.read_table("lookupTable_variantNo_"+str(variationCount)+".txt", sep=" ");
    genotype_info= pd.read_table("data\vcf_sample.txt", sep = "\t")
# sort lookup table 
    lookup_table = lookup_table.sort_values(by=['variant_id_1'])    

    lookup_table = lookup_table.reset_index(drop= True)  #make indexes start at 0

    
    prediction_matrix_haplotype1= pd.DataFrame()
    prediction_matrix_haplotype2= pd.DataFrame()

    hap1_individual_genotype_lst = []
    hap2_individual_genotype_lst = []
    temp = pd.DataFrame();
    
    pred_index =1 
    individuals = genotype_info.columns.tolist()[individual_starting_column:]
    individual_info_hap1 = pd.DataFrame(columns = individuals)
    individual_info_hap2 = pd.DataFrame(columns = individuals)

    genotype_info_hap1 = pd.DataFrame(columns = individuals)
    genotype_info_hap2 = pd.DataFrame(columns = individuals)

    for index, row in lookup_table.iterrows():
    
        hap1_individual_genotype_lst.clear()
        hap2_individual_genotype_lst.clear()
        columns = pd.Series()
        columns = columns.append(pd.Series({'gene_id':row['gene_id']}))
        for i in range(1,variationCount+1):
            var= 'variant_id_' + str(i)
            columns = columns.append(pd.Series({var:row[var]}))
            temp = columns.to_frame().T 
            key = row['gene_id']+ "_"+row[var]
            print (key)
        
# find the gene in genotype file
        
            genotype_index = list(np.where(genotype_info["gene_id_ID"] == key))[0]
        
            if (len(genotype_index)==0):     # chrX won't be found
                continue
#print (genotype_index[0]);
            hap_ind = 0
        
            for ind in individuals:
            
# make genotypes            
                genotype_both_haps= genotypeOfHaplotypes(genotype_info.loc[genotype_index[0],ind])
            
                if (i>1):
                    hap1_individual_genotype_lst.insert(hap_ind, hap1_individual_genotype_lst[hap_ind]+str(genotype_both_haps[0]) + ", ")
                    del hap1_individual_genotype_lst[hap_ind + 1]
                    hap2_individual_genotype_lst.insert(hap_ind, hap2_individual_genotype_lst[hap_ind]+str(genotype_both_haps[1]) + ", ")
                    del hap2_individual_genotype_lst[hap_ind + 1]
                else:
                    hap1_individual_genotype_lst.insert(hap_ind, str(genotype_both_haps[0]) + ", ")
                    hap2_individual_genotype_lst.insert(hap_ind, str(genotype_both_haps[1]) + ", ")
                hap_ind = hap_ind+1
   
    
    
    
    
    
    
        prediction_matrix_haplotype1= prediction_matrix_haplotype1.append(temp,ignore_index=True)
        prediction_matrix_haplotype2= prediction_matrix_haplotype2.append(temp,ignore_index=True)
    
        i = 0
        for genotype in hap1_individual_genotype_lst:
        
            genotype_info_hap1.at[index, individuals[i]] =  genotype[:-2]
            if ("2" in genotype[:-2] ):
# non-phased data
                individual_info_hap1.at[index, individuals[i]] = "NA"
        
            else:
# find the genotype in lookup table
                individual_info_hap1.at[index, individuals[i]] =  row[genotype[:-2]]
            i = i+1
    
        i=0;    
        for genotype in hap2_individual_genotype_lst:
        
            genotype_info_hap2.at[index, individuals[i]] =  genotype[:-2]
            if ("2" in genotype[:-2] ):
                individual_info_hap2.at[index, individuals[i]] = "NA"
        
            else:
        
                individual_info_hap2.at[index, individuals[i]] =  row[genotype[:-2]]
            i = i+1
        
        
        
    output_hap1 = pd.concat([prediction_matrix_haplotype1, individual_info_hap1],axis = 1, sort = False)
    output_hap2 = pd.concat([prediction_matrix_haplotype2, individual_info_hap2],axis = 1, sort = False)

    output_genotype_hap1 = pd.concat([prediction_matrix_haplotype1, genotype_info_hap1],axis = 1, sort = False)
    output_genotype_hap2 = pd.concat([prediction_matrix_haplotype2, genotype_info_hap2],axis = 1, sort = False)

#outputs
    output_hap1.to_csv('ASE_prediction_hap1_Variant_Num_'+str(variationCount)+".csv")
    output_hap2.to_csv('ASE_prediction_hap2_Variant_Num_'+str(variationCount)+".csv") 
    output_genotype_hap1.to_csv('genotype_hap1_Variant_Num_'+str(variationCount)+".csv")
    output_genotype_hap2.to_csv('genotype_hap2_Variant_Num_'+str(variationCount)+".csv")    


